# Introduction to Deep Learning with Keras
## Using the Keras API to Implement Deep Learning in Python

[Keras](https://keras.io/) is a high-level neural networks API, capable of running on top of [__Tensorflow__](https://github.com/tensorflow/tensorflow), [__Theano__](https://github.com/Theano/Theano), and [CNTK](https://github.com/Microsoft/cntk). It enables fast experimentation through a high level, user-friendly, modular and extensible API. Keras can also be run on both CPU and GPU.

Keras was developed and is maintained by [Francois Chollet](https://twitter.com/fchollet?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor) and is part of the Tensorflow core, which makes it Tensorflow's __preferred__ high-level API.

This tutorial explains how to use Keras for deep learning.

In this tutorial, we will go over the basics of Keras including the two most used Keras models (Sequential and Functional), the core layers as well as some preprocessing functionalities.

## Table of Contents
- [Keras Installation](#Keras-Installation)
- [Loading the Dataset](#Loading-the-Dataset)
- [Creating a Sequential Model](#Creating-a-sequential-model-in-Keras)
- [Dissecting the NN](#Dissecting-the-NN)
- [Creating a model with the Functional API](#Creating-a-model-with-the-Functional-API)
- [Compile a model](#Compile-a-model)
- [Augmenting Image Data](#Augmenting-Image-data)
- [Fit a Model](#Fit-a-model)
- [Conclusion](#Conclusion)
- [Exercise/Assignment](#Exercise/Assignment)

## Keras Installation

Keras is a front-end API - which means it handles user interation (I/O) and requires a working deep learning backend. Among the three we listed above, we are going to use __TensorFlow__ (TF) as the backend in this tutorial. TF is also the default backend for Keras.

You can use following command to install __TF__:
```bash
pip install tensorflow
```

Refer to this [link](https://www.tensorflow.org/install/) regarding TF installation.

After installing TF, you can use following command to install __Keras__:
```bash
pip install keras
```
or
```bash
conda install keras
```

Refer to this [link](https://keras.io/#installation) regarding Keras installation.

__NOTE__: If you are going to use TF as the backend of Keras, you do not have to configure the link. Otherwise (e.g. using Theano as the backend), you will need to change the backend in the `keras.json` file.

For the purpose of this tutorial, both TF and Keras are pre-installed on your instances.

## Loading the Dataset

Keras provides [seven different datasets](https://keras.io/datasets/), which can be loaded in using Keras directly. These include image datasets as well as a [house price](http://lib.stat.cmu.edu/datasets/boston) and a [movie review](https://ai.stanford.edu/~amaas/data/sentiment/) datasets.

In this article, we will use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/), which contains 70000 28x28 grayscale images with 10 different classes (`0`-`9`). Keras splits it in a __training set__ with `60000` instances and a __testing set__ with `10000` instances.


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
## Look at one of the instances
x_train[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  

In [ ]:
# We can also look at the dimensionalities of the instances
x_train[0].shape

(28, 28)

Depending on your version of Keras, you may see a message showing that you are `Using TensorFlow backend`.

From results above, we know that the images are stored in `28 x 28 ndarray` objects.

To feed the images to a [convolutional neural network (CNN)](http://cs231n.github.io/convolutional-networks/) we transform the dataframe to four dimensions. This can be done using numpys reshape method. We will also transform the data into floats and normalize it.

__NOTE 1__: For most part of deep learning, we can only handle floats or integers.

__NOTE 2__: Can you guess why we want to normalize the data here?

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

We can understand the first part of above code block, where we set the datatype as `float` and normalize the value to `[0,1]` range. But what does the second part do?

Actually, neural networks treat data as tensors (hence the name TensorFlow) - and tensors are multi-dimensional vectors. In this case, we convert the images to add a value `1` as the last dimension of the tensors. Particularly in image classification (which is what we are doing here), the value of `1` indicates one color channel (i.e. __grayscale images__) - if the images are colored, we can use a color channel of `3`.



In [ ]:
x_train[0].shape

(28, 28, 1)

We will also transform our labels into an [one-hot encoding](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f) using the `to_categorical()` method from Keras.

__NOTE__: in data mining, OHE is not required - but in most cases of deep learning, we will perform OHE on the categorical target. The reason is by default neural nets treat all values as continuous (`float`).

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# let's look at our encoded target
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

### YOUR TURN HERE - ANSWER THE QUESTION
Q: What digit does the instance `y_train[0]` correspond to?

5

# Creating a sequential model in Keras
The easiest way of creating a model in Keras is by using the sequential API, which lets you stack one layer after the other. The problem with the sequential API is that it doesn’t allow models to have multiple inputs or outputs, which are needed for some problems.

Nevertheless, the sequential API is a perfect choice for most problems.

To create a CNN we only need to create a `Sequential` object and use the `add` function to add layers.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=x_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(10, activation='softmax'))

## Dissecting the NN

Now you see we used five types of layers from Keras, namely `Conv2D`, `MaxPool2D`, `Dense`, `Flatten` and `Dropout`. Let's briefly look at what they do, respectively.

### Conv2D and Maxpool2D

These two are a pair. Convolution (`Conv2D`) in general means we take the 5x5x3 filter and slide it over the complete image and along the way take the dot product between the filter and chunks of the input image.

Original Image and Sliding Window
<img src= 'https://cdn-images-1.medium.com/max/800/1*FUEkm0JghT3ab8P7p9c5Qg.png' />

Convolution
<img src= 'https://cdn-images-1.medium.com/max/800/1*3sfzVenrdS5MWGsmSCbx3A.png' />
<img src = 'https://cdn-images-1.medium.com/max/800/1*mcBbGiV8ne9NhF3SlpjAsA.png' />

You may wonder how do we convert a section of image (via sliding window) to a certain number - that is the magic of pooling. The most popular pooling technique is Maxpooling (`Maxpool2D`).

(Max-) Pooling
<img src = 'https://cdn-images-1.medium.com/max/800/1*jU_Mp73fXzh9_ffvtnbrDQ.png' />

If you wish to know more regarding Convolution and Pooling, [this article](https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/) or [this article](https://machinelearningmastery.com/pooling-layers-for-convolutional-neural-networks/) can be good starting point.

Another point worthing noting here is the activation function. Activation function decides, whether a neuron should be __activated__ (output a `True` or `1` value) or not (output a `False` or `0` value) by calculating weighted sum and further adding bias with it. The purpose of the activation function is to introduce __non-linearity__ into the output of a neuron. In this tutorial we used the __Rectified Linear Unit (ReLU)__ activation function in the `conv2D` layers - given any value $x$, if $x > 0$ then output $x$; otherwise output $0$.

<img src = 'https://cdn-images-1.medium.com/max/800/1*DfMRHwxY1gyyDmrIAd-gjQ.png' />

We use a `softmax` activation to get the final output of the NN. Generally, we use __softmax__ activation instead of __sigmoid__ with the cross-entropy loss because softmax activation distributes the probability throughout each output node. If the problem is a binary classification, using sigmoid is same as softmax. For multi-class classification use `softmax` with `cross-entropy`.


<img serc = 'https://cdn-images-1.medium.com/max/800/1*670CdxchunD-yAuUWdI7Bw.png' />

More discussions regarding ReLU and other activation function can be found [here](https://medium.com/the-theory-of-everything/understanding-activation-functions-in-neural-networks-9491262884e0).

### Flatten and Dense

After maxpooling, we have pooled feature maps from images. The flatten layer, as the name suggests, flatten our pooled feature map into a column like in the image below.

<img src = 'https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/73_blog_image_1.png' />

A dense layer represents a matrix vector multiplication. (assuming your batch size is 1) The values in the matrix are the trainable parameters which get updated during backpropagation.

So you get a m dimensional vector as output. A dense layer thus is used to change the dimensions of your vector. Mathematically speaking, it applies a rotation, scaling, translation transform to your vector.

### Dropout

A dropout layer is used for regularization where you randomly set some of the dimensions of your input vector to be zero with probability `𝑘𝑒𝑒𝑝_𝑝𝑟𝑜𝑏`. A dropout layer does not have any trainable parameters (i.e. nothing gets updated during backward pass of backpropagation). To ensure that expected sum of vectors fed to this layer remains the same if no dropout was applied, the remaining dimensions which are not set to zero are scaled by $\frac{1}{𝑘𝑒𝑒𝑝\_𝑝𝑟𝑜𝑏}$.

### IMPORTANT
Remember in the beginning we said that Keras is a high-level API that builds on the backend like TF? You must wonder why we need such an API - the answer is Keras makes our life easier. For instance, a similar (and much __simpler__) NN would be written in TF like below:
```python
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
```

### Returning to Our Sequential Model
The code above first of creates a `Sequential` object and adds a few convolutional, maxpooling and dropout layers. It then flattens the output and passes it two a last dense and dropout layer before passing it to our output layer. If you aren’t confident build a CNN check out [this great tutorial](http://cs231n.github.io/convolutional-networks/).

The sequential API also supports another syntax where the layers are passed to the constructor directly.

```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

model = Sequential([
    Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=x_train.shape[1:]),
    Conv2D(filters=32, kernel_size=(5,5), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(rate=0.25),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(rate=0.25),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(rate=0.5),
    Dense(10, activation='softmax')
])
```


# Creating a model with the Functional API
Alternatively, the functional API allows you to create the same models but offers you more flexibility at the cost of simplicity and readability.

The functional API allows you to create models that have a lot more __flexibility__ as you can easily define models where layers connect to more than just the previous and next layers. In fact, you can connect layers to (literally) any other layer. As a result, creating __complex network__s such as siamese networks and residual networks become possible. The functional API is also useful in [transfer learning](https://machinelearningmastery.com/transfer-learning-for-deep-learning/).

It can be used with multiple input and output layers as well as shared layers, which enables you to build really complex network structures.

When using the functional API we always need to pass the previous layer to the current layer. It also requires the use of an input layer.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
# from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, Input

inputs = Input(shape=x_train.shape[1:])

x = Conv2D(filters=32, kernel_size=(5,5), activation='relu')(inputs)
x = Conv2D(filters=32, kernel_size=(5,5), activation='relu')(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.25)(x)

x = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(x)
x = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.25)(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(rate=0.5)(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)

__NOTE__: as a rule of thumb, if I am training a preliminary/baseline model, I will use the sequential API; if I am training a model for finetuning into a final model, I will use the functional API.

# Compile a model
Before we can start training our model we need to configure the learning process. For this, we need to specify an __optimizer__, a __loss function__ and optionally some __metrics__ like _accuracy_.

The [loss function](https://www.youtube.com/watch?v=IVVVjBSk9N0) is a measure on how good our model is at achieving the given objective. [Here](https://keras.io/losses/) is a list of all loss functions supported by Keras.

An [optimizer](https://towardsdatascience.com/a-quick-guide-to-neural-network-optimizers-with-applications-in-keras-e4635dd1cca4) is used to minimize the loss(objective) function by updating the weights using the gradients. [Here](https://keras.io/optimizers/) is a list of all optimizers supported by Keras.

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# We should always check if the model architecture fits what we want
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 24, 24, 32)        832       
                                                                 
 conv2d_5 (Conv2D)           (None, 20, 20, 32)        25632     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 10, 10, 32)        0         
                                                                 
 conv2d_6 (Conv2D)           (None, 8, 8, 64)          18496     
                                                             

### YOUR TURN HERE - ANSWER THE QUESTION
Use your own words, along with some research on the `categorical_entropy`, explain why we use it, rather than any other loss function supported by Keras.

We use categorical cross-entropy because of its suitability for classification problems, gradient efficiency, and compatibility with softmax activation. It also measures the dissimilarity between prediceted class probabilities and true class labels.

### YOUR TURN HERE - ANSWER THE QUESTION
Explain what is the difference between a loss function and an evaluation metric.

Loss function is used during training to optimize the model's parameters by measuring the error between the predicted and actual values. On the other hand, evaluation metrics are used after training to assess the overall performance of the model and determine how well it will solve the specific problem that we have

# Augmenting Image data
Augmentation is an optional step of creating more data from existing ones. As you know, we are always in demand for more labelled training data - that is in particular true with NN/DL since we need more data to train the network.

For images you can to little transformations like rotating the image, zooming into the image, adding noise and many more.

This helps to make the model more robust and solves the problem of having not enough data. Keras has a method called ImageDataGenerator which can be used for augmenting images.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
  rotation_range=10,
  zoom_range=0.1,
  width_shift_range=0.1,
  height_shift_range=0.1
)

This `ImageDataGenerator` will create new images that have been rotated, zoomed in or out, and shifted in width and height.

# Fit a model
Now that we defined and compiled our model it’s ready for training. To train a model we would normally use the `fit` method but because we are using a datagenerator we will use `fit_generator` and pass it our generator, X data, y data as well as the [number of epochs and the batch size](https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9). We will also pass it a _validation set_ so we can monitor the __loss__ and __accuracy__ on both sets as well as `steps_per_epoch` which is required when using a generator and is just set to the length of the training set divided by the `batch_size`.

In [ ]:
epochs = 5
batch_size = 64
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                              validation_data=(x_test, y_test), steps_per_epoch=x_train.shape[0]//batch_size)

Epoch 1/5
937/937 [==============================] - ETA: 0s - batch: 468.0000 - size: 63.9658 - loss: 0.3965 - acc: 0.8718

/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


937/937 [==============================] - 252s 268ms/step - batch: 468.0000 - size: 63.9658 - loss: 0.3965 - acc: 0.8718 - val_loss: 0.0452 - val_acc: 0.9850
Epoch 2/5
937/937 [==============================] - 250s 266ms/step - batch: 468.0000 - size: 63.9658 - loss: 0.1281 - acc: 0.9621 - val_loss: 0.0317 - val_acc: 0.9898
Epoch 3/5
937/937 [==============================] - 247s 263ms/step - batch: 468.0000 - size: 63.9658 - loss: 0.0965 - acc: 0.9724 - val_loss: 0.0238 - val_acc: 0.9917
Epoch 4/5
937/937 [==============================] - 248s 265ms/step - batch: 468.0000 - size: 63.9658 - loss: 0.0791 - acc: 0.9772 - val_loss: 0.0197 - val_acc: 0.9937
Epoch 5/5
937/937 [==============================] - 236s 252ms/step - batch: 468.0000 - size: 63.9658 - loss: 0.0700 - acc: 0.9796 - val_loss: 0.0213 - val_acc: 0.9930


The training history showed above contain rich information that we can use. For instance, it can provide important information regarding overfitting of a model. There are a few rules that we can use to determine if a NN is overfitted.

__Primary Rule__: if the validation evaluation metric (e.g. `val_acc`) is not improving over epochs, then the NN is likely overfitted.

__Seconday Rule__: if the training evaluation metric (e.g. `acc`) is constantly significantly higher than the validation evaluation metric (e.g. `val_acc`), then the NN is likely overfitted.

If a NN overfits, you can apply steps like _early stopping_ in training - [this](https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/) would be a good intro for _early stopping_.

To better monitor the training history, we can visualize our training and testing accuracy and loss for each epoch so we can get intuition about the performance of our model. The accuracy and loss over epochs are saved in the history variable we got whilst training and we will use Matplotlib to visualize this data.

We first visualize the __evaluation metric__ (_accuracy_) between training and validation sets.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

plt.plot(history.history['acc'], label='training accuracy')
plt.plot(history.history['val_acc'], label='testing accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc='lower right')

Then we can visualize the loss between training and validation sets.

In [ ]:
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='testing loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()

From above graphs, if you observe _accuracy_ is not improving, or _loss_ is not decreasing, on the __validation set__, the NN is likely overfitting.

In the graphs above we can see that our model isn’t overfitting as well as that we could train more epochs because the validation loss is still decreasing. You can also use some built-in tool like [TensorBoard](https://fizzylogic.nl/2017/05/08/monitor-progress-of-your-keras-based-neural-network-using-tensorboard/) for monitoring the training process.

That is all for this tutorial.

## Conclusion

In this tutorial, you are introduced to a high-level deep learning API, namely Keras. Keras can help build NNs in a easy way. It also provides a lot of built-in functions and methods help evaluate and monitor the training process. Even though we did not talk about it, Keras can also help with the _deployment_ of a trained/evaluated NN.

If you would like to know more regarding Keras, please refer to [this article](https://www.pyimagesearch.com/2018/09/10/keras-tutorial-how-to-get-started-with-keras-deep-learning-and-python/).

# Exercise/Assignment

Please complete [this article on Datacamp](https://www.datacamp.com/community/tutorials/deep-learning-python) to further understanding Keras. Unlike the image data we used in the tutorial, this article focuses on traditional tabular data. You are __REQUIRED__ to provide screenshots of all code snippets need completion (a total of __15__, with __RESULTS__) as your submission. Please paste all screenshots into a __SINGLE__ document and submit it in **Google Classroom**, along with this notebook.